In [ ]:
from pylab import *
import random as rd
import numpy as np
import networkx as nx

In [ ]:
def malla_intercrop_row(F,C,r):
    
    #F_filas de cultivo susceptible + (F-1) filas de cultivo inmune
    
    g = nx.Graph()
    
    cont = 0
    
    for i in range(0,int(2*F-1)):
        
        for j in range(0,int(C)):

            g.add_node(cont)
            g.node[cont]['x'] = j*0.3
            g.node[cont]['y'] = i*0.375
            
            cont += 1

    for i in g.nodes_iter():

        for j in g.nodes_iter():

            if i != j and g.has_edge(i,j)==False and\
            (g.node[i]['x'] - g.node[j]['x'])**2.0 + (g.node[i]['y']-g.node[j]['y'])**2.0 <= r**2.0:

                g.add_edge(i,j)
                
    return g


def malla_monocrop(F,C,r):
    
    #F_filas de cultivo susceptible
    
    g = nx.Graph()
    
    cont = 0
    
    for i in range(0,int(F)):
        
        for j in range(0,int(C)):

            g.add_node(cont)
            g.node[cont]['x'] = j*0.3
            g.node[cont]['y'] = i*0.75
            
            cont += 1

    for i in g.nodes_iter():

        for j in g.nodes_iter():

            if i != j and g.has_edge(i,j)==False and\
            (g.node[i]['x'] - g.node[j]['x'])**2.0 + (g.node[i]['y']-g.node[j]['y'])**2.0 <= r**2.0:

                g.add_edge(i,j)
                
    return g



In [ ]:
def monocrop(F,C,fraction_infected_t0,inf_border):
    
    global g
    
    crop_inmune = []
    crop_suscep = []
    
    for i in g.nodes_iter():
            
        crop_suscep.append(i)
            
    if inf_border == False:
        
        crop_infect = rd.sample(crop_suscep,int(fraction_infected_t0*len(crop_suscep)))
        
    else:
        
        borde_rec = []
        
        for i in g.nodes_iter():
            
            if g.node[i]['x'] == 0 or g.node[i]['x']==6.0 or g.node[i]['y'] == 0 or g.node[i]['y']==6.0:
                
                borde_rec.append(i)
            
        borde_sus = [i for i in borde_rec if i not in crop_inmune]
        crop_infect = rd.sample(borde_sus,int(1))
    
    return crop_inmune, crop_suscep, crop_infect

def row_khan(F,C,fraction_infected_t0,inf_border):
    
    global g

    
    crop_inmune = []
    crop_suscep = []
    
    cont = 0
    
    for i in range(0,int(2*F-1)):
            
        for j in range(0,int(C)):
            
            if i%2==0:
                
                crop_suscep.append(cont)
                cont += 1
                
            else:
                
                crop_inmune.append(cont)
                cont += 1
    
    if inf_border == False:
        
        crop_infect = rd.sample(crop_suscep,int(fraction_infected_t0*len(crop_suscep)))
        
    else:
        
        borde_rec = []
        
        for i in g.nodes_iter():
            
            if g.node[i]['x'] == 0 or g.node[i]['x']==6.0 or g.node[i]['y'] == 0 or g.node[i]['y']==6.0:
                
                borde_rec.append(i)
                   
        borde_sus = [i for i in borde_rec if i not in crop_inmune]
        crop_infect = rd.sample(borde_sus,int(1))
    
    return crop_inmune, crop_suscep, crop_infect

In [ ]:
# SIR DYNAMIC WITHOUT LONG-RANGE INTERACTIONS

def q_i_SIN(nodo,lista_p_i):
    
    global g,beta_cons

    aux = 1.0
    
    for i in g.neighbors_iter(nodo):
        
        aux *= (1.0 - beta_cons*lista_p_i[i])
        
    return 1.0 - aux

def Markov_evolution_SIR_SIN_cuad(crop_inmune, crop_suscep, crop_infect):
    
    global g,beta_cons,mu_cons,tmax,lista_infectados_ini,\
    lista_recuperados_ini,lista_recuperados_final
    
    N0 = len(g.nodes())
    
    lista_s_i = [] #susceptibles
    lista_p_i = [] #infectados
    lista_rho_i = [] #recuperados
    
    for i in g.nodes_iter():

        lista_s_i.append(1.0) #susceptibles
        lista_p_i.append(0.0) #infectados
        lista_rho_i.append(0.0) #recuperados
        
    N0 = len(g.nodes())
    
    cant_inmunes_t0 = len(crop_inmune)
    lista_infectados_ini = crop_infect[:]
    lista_recuperados_ini = crop_inmune[:]
    
    for i in crop_inmune:
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 0.0
        lista_rho_i[i] = 1.0
        
    for i in crop_infect :
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 1.0
        lista_rho_i[i] = 0.0

    
    amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
    amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
    amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))

    
    t = 0
    terminar = False
    
    while t <= tmax and terminar==False:
    
        new_s_i = []
        new_p_i = []
        new_rho_i = []

        for a in g.nodes_iter():

            
            if lista_rho_i[a] == 1.0:
                
                new_s_i.append(0.0)
                new_p_i.append(0.0)
                new_rho_i.append(1.0)
                
            else:  

                q_i_nodo = q_i_SIN(a,lista_p_i)

                aux_new_p_i = lista_p_i[a]*(1.0 - mu_cons) + (1.0 - lista_p_i[a] - lista_rho_i[a])*q_i_nodo
                aux_new_rho_i = lista_rho_i[a] + mu_cons*lista_p_i[a]

                new_s_i.append(1.0 - aux_new_p_i - aux_new_rho_i)
                new_p_i.append(aux_new_p_i)
                new_rho_i.append(aux_new_rho_i)
        
        if max(new_p_i) < 1e-6:

            terminar = True

        lista_s_i = new_s_i[:]
        lista_p_i = new_p_i[:]
        lista_rho_i = new_rho_i[:]

        
        amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
        amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
        amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))

        t += 1
        
    
    #print 't_sim:',t
    
    lista_recuperados_final=[i for i in g.nodes_iter() if lista_rho_i[i] > lista_s_i[i]]
    
    return 1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0)


In [ ]:
def matriz_distancia():
    
    global g
    
    dic_shortest = nx.shortest_path(g)

    M_nueva = []

    for i in range(0,len(g.nodes())):

        fila_aux = []

        fila = dic_shortest.get(i)

        for k in range(0,len(g.nodes())):

            fila_aux.append(len(fila.get(k))-1.0)

        M_nueva.append(fila_aux)
    
    return M_nueva


In [ ]:
def matriz_alargamiento():
    
    global g,crop_inmune
    
    dic_shortest = nx.shortest_path(g)

    M_nueva = []

    for i in range(0,len(g.nodes())):

        fila_aux = []

        fila = dic_shortest.get(i)

        for k in range(0,len(g.nodes())):
            
            camino_ik = fila.get(k)
            
            aux = 0.0
            
            for p in camino_ik:
                
                if p in crop_inmune:
                    
                    aux += 1.0

            fila_aux.append(aux)

        M_nueva.append(fila_aux)
    
    return M_nueva

In [ ]:
# SIR DYNAMIC WITHOUT TRAP CROPS

def q_i_MELLIN(nodo,lista_p_i,s):
    
    global g,beta_cons,M_distancia

    aux = 1.0
    
    if s >= 100.0:
        
        return q_i_SIN(nodo,lista_p_i)
    
    else:
        
    
        for i in g.nodes_iter():
            
            if i != nodo:

                aux *= (1.0 - 1.0*beta_cons*lista_p_i[i]/((M_distancia[nodo][i])**s))
        
        return (1.0 - aux)
    


def Markov_evolution_SIR_MELLIN_cuad_time(s,crop_inmune, crop_suscep, crop_infect):
    
    global g,beta_cons,mu_cons,tmax,lista_infectados_ini,\
    lista_recuperados_ini,lista_recuperados_final,M_distancia
    
    N0 = len(g.nodes())
    
    lista_s_i = [] #susceptibles
    lista_p_i = [] #infectados
    lista_rho_i = [] #recuperados
    
    for i in g.nodes_iter():

        lista_s_i.append(1.0) #susceptibles
        lista_p_i.append(0.0) #infectados
        lista_rho_i.append(0.0) #recuperados
        
    N0 = len(g.nodes())
    
    cant_inmunes_t0 = len(crop_inmune)
    lista_infectados_ini = crop_infect[:]
    lista_recuperados_ini = crop_inmune[:]


    for i in crop_inmune:
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 0.0
        lista_rho_i[i] = 1.0
        
    for i in crop_infect :
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 1.0
        lista_rho_i[i] = 0.0
    
    amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
    amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
    amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))   
    
    t = 0
    terminar = False
    
    while t <= tmax and terminar==False:
    
        new_s_i = []
        new_p_i = []
        new_rho_i = []

        for a in g.nodes_iter():
            
            if g.node[a]['buscar'] == True: #lista_p_i[a] < lista_rho_i[a]:
                
                g.node[a]['time'] = t
                
                if 1.0 - lista_p_i[a] - lista_rho_i[a] < lista_rho_i[a]:
                
                    g.node[a]['buscar'] = False
            
            if s >= 100:
            
                q_i_nodo = q_i_SIN(a,lista_p_i)
                
            else:
                
                q_i_nodo = q_i_MELLIN(a,lista_p_i,s)

            aux_new_p_i = lista_p_i[a]*(1.0 - mu_cons) + (1.0 - lista_p_i[a] - lista_rho_i[a])*q_i_nodo
            aux_new_rho_i = lista_rho_i[a] + mu_cons*lista_p_i[a]

            new_s_i.append(1.0 - aux_new_p_i - aux_new_rho_i)
            new_p_i.append(aux_new_p_i)
            new_rho_i.append(aux_new_rho_i)

        
        if max(new_p_i) < 1e-6:

            terminar = True

        lista_s_i = new_s_i[:]
        lista_p_i = new_p_i[:]
        lista_rho_i = new_rho_i[:]
        
        amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
        amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
        amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))

        t += 1

    
    lista_recuperados_final=[i for i in g.nodes_iter() if lista_rho_i[i] > lista_s_i[i]]
    
    return 1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0)

In [ ]:

# SIR DYNAMIC WITH TRAP CROPS


def q_i_MELLIN_SEXY3_mod(nodo,lista_p_i,s,gamma):
    
    global g,beta_cons,M_distancia,M_alargamiento,crop_suscep

    aux = 1.0
    
    for i in crop_suscep:

        if i != nodo:
                
            if M_alargamiento[nodo][i] == 0.0:

                aux *= (1.0 - 1.0*beta_cons*lista_p_i[i]/((M_distancia[nodo][i])**s))
                
            else:
                
                aux *= (1.0 - 1.0*beta_cons*lista_p_i[i]/((M_distancia[nodo][i])**(gamma*s)))
 
    return (1.0 - aux)
    

def Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(s,crop_inmune, crop_suscep, crop_infect):
    
    global g,beta_cons,mu_cons,tmax,lista_infectados_ini,\
    lista_recuperados_ini,lista_recuperados_final,M_distancia,M_alargamiento,rho_est,gamma
    
    N0 = len(g.nodes())
    
    lista_s_i = [] #susceptibles
    lista_p_i = [] #infectados
    lista_rho_i = [] #recuperados
    
    for i in g.nodes_iter():

        lista_s_i.append(1.0) #susceptibles
        lista_p_i.append(0.0) #infectados
        lista_rho_i.append(0.0) #recuperados
        
    N0 = len(g.nodes())
    
    cant_inmunes_t0 = len(crop_inmune)
    lista_infectados_ini = crop_infect[:]
    lista_recuperados_ini = crop_inmune[:]
    


    for i in crop_inmune:
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 0.0
        lista_rho_i[i] = 1.0
        
    for i in crop_infect :
        
        lista_s_i[i] =  0.0
        lista_p_i[i] = 1.0
        lista_rho_i[i] = 0.0

    
    amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
    amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
    amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))
    
    
    t = 0
    terminar = False
    
    while t <= tmax and terminar==False:
    
        new_s_i = []
        new_p_i = []
        new_rho_i = []

        for a in g.nodes_iter():
            
            if g.node[a]['buscar'] == True: #lista_p_i[a] < lista_rho_i[a]:
                
                g.node[a]['time'] = t
                
                if 1.0 - lista_p_i[a] - lista_rho_i[a] < lista_rho_i[a]:
                
                    g.node[a]['buscar'] = False
            
            if s >= 100:
            
                q_i_nodo = q_i_SIN(a,lista_p_i)
                
            else:
                
                q_i_nodo = q_i_MELLIN_SEXY3_mod(a,lista_p_i,s,gamma) #q_i_MELLIN(a,lista_p_i,s)

            aux_new_p_i = lista_p_i[a]*(1.0 - mu_cons) + (1.0 - lista_p_i[a] - lista_rho_i[a])*q_i_nodo
            aux_new_rho_i = lista_rho_i[a] + mu_cons*lista_p_i[a]

            new_s_i.append(1.0 - aux_new_p_i - aux_new_rho_i)
            new_p_i.append(aux_new_p_i)
            new_rho_i.append(aux_new_rho_i)

        
        if max(new_p_i) < 1e-6:

            terminar = True

        lista_s_i = new_s_i[:]
        lista_p_i = new_p_i[:]
        lista_rho_i = new_rho_i[:]
        
        amount_inf.append(1.0*sum(lista_p_i)/(N0-cant_inmunes_t0))
        amount_sus.append(1.0*sum(lista_s_i)/(N0-cant_inmunes_t0))
        amount_rec.append(1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0))

        t += 1
        
    
    
    return 1.0*(sum(lista_rho_i)-cant_inmunes_t0)/(N0-cant_inmunes_t0)

In [ ]:
######################################################
#KHAN et al. INTERCROPS (Table 2)
######################################################
#DATA GENERATION
######################################################

In [ ]:
#First Column (2001)

In [ ]:
F = 9
C = 21
r = 0.75
fraction_infected_t0 = 0.03
inf_border = True
gamma = 2.0
#Primera columna Khan
mu_cons = 0.5

tmax = 10
mell = 3.3

rep = 100

beta_desmodium = 0.1165
beta_monocrop = 0.1165
beta_cowpea = 0.0995

###########################################################################

beta_cons = beta_desmodium


g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

inter_colum1_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'fila'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    inter_colum1_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################

beta_cons = beta_cowpea

g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

cowpea_colum1_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'cow'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_cuad_time(mell,crop_inmune, crop_suscep, crop_infect)
    cowpea_colum1_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################
beta_cons = beta_monocrop

g = malla_monocrop(F,C,r)

crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)


M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

mono_colum1_TRAMPA = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i


    crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)
    

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    mono_colum1_TRAMPA.append(1.0-round(amount_sus[-1],4))
    
############################################################################################


In [ ]:
print 'col. 1: beta =', beta_desmodium,'(I+R)_Row =',sum(inter_colum1_TRAMPA_fila)*1.0/rep
print 'col. 1: beta =', beta_cowpea,'(I+R)_Cow =',sum(cowpea_colum1_TRAMPA_fila)*1.0/rep
print 'col. 1: beta =', beta_monocrop,'(I+R)_Monocrop =',sum(mono_colum1_TRAMPA)*1.0/rep

In [ ]:
#Second Column (2002)

In [ ]:
F = 9
C = 21
r = 0.75
fraction_infected_t0 = 0.03
inf_border = True

#Segunda columna
mu_cons = 0.5
beta_cons = 0.151 #Cowpea #0.151
tmax = 10
mell = 3.3

rep = 100

beta_desmodium = 0.151
beta_monocrop = 0.151
beta_cowpea = 0.1345

###########################################################################

beta_cons = beta_desmodium


g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

inter_colum2_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'fila'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    inter_colum2_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################

beta_cons = beta_cowpea

g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

cowpea_colum2_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'cow'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_cuad_time(mell,crop_inmune, crop_suscep, crop_infect)
    cowpea_colum2_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################
beta_cons = beta_monocrop

g = malla_monocrop(F,C,r)

crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)


M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

mono_colum2_TRAMPA = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i


    crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)
    

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    mono_colum2_TRAMPA.append(1.0-round(amount_sus[-1],4))
    
############################################################################################

In [ ]:
print 'col. 2: beta =', beta_desmodium,'(I+R)_Row =',sum(inter_colum2_TRAMPA_fila)*1.0/rep
print 'col. 2: beta =', beta_cowpea,'(I+R)_Cow =',sum(cowpea_colum2_TRAMPA_fila)*1.0/rep
print 'col. 2: beta =', beta_monocrop,'(I+R)_Monocrop =',sum(mono_colum2_TRAMPA)*1.0/rep

In [ ]:
#Third Column (2003)

In [ ]:
F = 9
C = 21
r = 0.75
fraction_infected_t0 = 0.03
inf_border = True

#Tercera columna
mu_cons = 0.5
beta_cons = 0.158#Cowpea# 0.158
tmax = 10
mell = 3.3


rep = 100

beta_desmodium = 0.1555
beta_cowpea = 0.1235
beta_monocrop = 0.1555

###########################################################################

beta_cons = beta_desmodium


g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

inter_colum3_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'fila'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    inter_colum3_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################

beta_cons = beta_cowpea

g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

cowpea_colum3_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'cow'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_cuad_time(mell,crop_inmune, crop_suscep, crop_infect)
    cowpea_colum3_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################
beta_cons = beta_monocrop

g = malla_monocrop(F,C,r)

crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)


M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

mono_colum3_TRAMPA = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i


    crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)
    

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    mono_colum3_TRAMPA.append(1.0-round(amount_sus[-1],4))
    
############################################################################################

In [ ]:
print 'col. 3: beta =', beta_desmodium,'(I+R)_Row =',sum(inter_colum3_TRAMPA_fila)*1.0/rep
print 'col. 3: beta =', beta_cowpea,'(I+R)_Cow =',sum(cowpea_colum3_TRAMPA_fila)*1.0/rep
print 'col. 3: beta =', beta_monocrop,'(I+R)_Monocrop =',sum(mono_colum3_TRAMPA)*1.0/rep

In [ ]:
#Fourth Column (2004)

In [ ]:
F = 9
C = 21
r = 0.75
fraction_infected_t0 = 0.03
inf_border = True

#Cuarta columna
mu_cons = 0.5
beta_cons = 0.1 #Cowpea #0.1
tmax = 10
mell = 3.3

rep = 100

beta_desmodium = 0.098
beta_cowpea = 0.086
beta_monocrop = 0.098
###########################################################################

beta_cons = beta_desmodium


g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

inter_colum4_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'fila'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    inter_colum4_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################

beta_cons = beta_cowpea

g = malla_intercrop_row(F,C,r)
crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

cowpea_colum4_TRAMPA_fila = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i,'cow'
    
    crop_inmune, crop_suscep, crop_infect = row_khan(F,C,fraction_infected_t0,inf_border)

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_cuad_time(mell,crop_inmune, crop_suscep, crop_infect)
    cowpea_colum4_TRAMPA_fila.append(1.-round(amount_sus[-1],4))


###########################################################################
###########################################################################
beta_cons = beta_monocrop

g = malla_monocrop(F,C,r)

crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)


M_alargamiento = matriz_alargamiento()
M_distancia = matriz_distancia()

mono_colum4_TRAMPA = []

for i in range(0,rep):
    
    if i % 30 == 0:
        print i


    crop_inmune, crop_suscep, crop_infect = monocrop(F,C,fraction_infected_t0,inf_border)
    

    amount_inf = []
    amount_sus = []
    amount_rec = []
    lista_infectados_ini = []
    lista_recuperados_ini = []
    lista_recuperados_final = []


    for i in g.nodes_iter():

        g.node[i]['time'] = 0
        g.node[i]['buscar'] = True

    Markov_evolution_SIR_MELLIN_SEXY_cuad3_time_mod(mell,crop_inmune, crop_suscep, crop_infect)
    mono_colum4_TRAMPA.append(1.0-round(amount_sus[-1],4))
    
############################################################################################

In [ ]:
print 'col. 4: beta =', beta_desmodium,'(I+R)_Row =',sum(inter_colum4_TRAMPA_fila)*1.0/rep
print 'col. 4: beta =', beta_cowpea,'(I+R)_Cow =',sum(cowpea_colum4_TRAMPA_fila)*1.0/rep
print 'col. 4: beta =', beta_monocrop,'(I+R)_Monocrop =',sum(mono_colum4_TRAMPA)*1.0/rep

In [ ]:
#DATA STORAGE

In [ ]:
from pandas import DataFrame

df = DataFrame({'01.Column1_Monocrop':mono_colum1_TRAMPA,\
                '02.Column1_Cowpea': cowpea_colum1_TRAMPA_fila,\
                '03.Column2_Monocrop':mono_colum2_TRAMPA,\
                '04.Column2_Cowpea': cowpea_colum2_TRAMPA_fila,\
                '05.Column3_Monocrop':mono_colum3_TRAMPA,\
                '06.Column3_Cowpea': cowpea_colum3_TRAMPA_fila,\
                '07.Column4_Monocrop':mono_colum4_TRAMPA,\
                '08.Column4_Cowpea': cowpea_colum4_TRAMPA_fila})
    
print df

df.to_excel('Fig2a_gamma2.xlsx', sheet_name='sheet1', index=False)

In [ ]:
from pandas import DataFrame

df = DataFrame({'01.Column1_Monocrop':mono_colum1_TRAMPA,\
                '02.Column1_Row': inter_colum1_TRAMPA_fila,\
                '03.Column2_Monocrop':mono_colum2_TRAMPA,\
                '04.Column2_Row': inter_colum2_TRAMPA_fila,\
                '05.Column3_Monocrop':mono_colum3_TRAMPA,\
                '06.Column3_Row': inter_colum3_TRAMPA_fila,\
                '07.Column4_Monocrop':mono_colum4_TRAMPA,\
                '08.Column4_Row': inter_colum4_TRAMPA_fila})
    
print df

df.to_excel('Fig2b_gamma2.xlsx', sheet_name='sheet1', index=False)